In [1]:
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image

from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
def pathJoin(*args):
    return os.path.abspath(os.path.join(*args))

class BaseDataset(Dataset):

    def __init__(self, directory, split='train', transforms=None):
        self.datapoints = defaultdict(list)
        self.split = split
        self.directory = pathJoin(directory, split)
        self.datapoints = self.loadDataset()
        self.transforms = transforms

    def __len__(self):
        return len(self.datapoints)

    def __getitem__(self, idx):
        datapoint = self.loadDatapoint(idx)
        return datapoint

    def loadDatapoint(self, idx):
        raise NotImplementedError('Function "loadDatapoint" is not implemented')

    def loadDataset(self, name):
        raise NotImplementedError('Function "loadDataset" is not implemented')

class ImageNetDataset(BaseDataset):

    def __init__(self, directory, split='train', transforms=None):
        super().__init__(directory, split, transforms)
        self.descriptions = self.loadDescriptions()
        self.classes = self.loadClasses()
        self.groundtruths = self.loadValidationGroundtruths() if split == 'val' else []

    def loadDatapoint(self, idx):
        filepath = self.datapoints[idx]
        image = Image.open(filepath).convert('RGB')
        if self.split == 'val':
            groundtruth = self.groundtruths[idx]
        elif self.split == 'train':
            groundtruth = self.classes.index(filepath.split('/').pop().split('_')[0])
        if self.transforms:
            image = self.transforms(image)
        return (filepath, image, groundtruth, self.descriptions[groundtruth])

    def loadDataset(self):
        datapoints = []

        dataset_file_list_filename = 'ilsvrc2012{}.txt'.format(self.split)
        dataset_file_list_path = os.path.join(self.directory, dataset_file_list_filename)

        with open(dataset_file_list_path, 'r') as dataset_file_list_file:
            for line in tqdm(dataset_file_list_file, total=sum(1 for line in open(dataset_file_list_path))):
                file_path = pathJoin(self.directory, self.sanitizeFilename(line))
                datapoints.append(file_path)
        
        return datapoints
    
    def sanitizeFilename(self, filename):
        return filename.replace('"', '').strip()

    def loadDescriptions(self):
        descriptions = []

        descriptions_filename = 'synsets_with_descriptions.txt'
        descriptions_path = pathJoin(self.directory, '..', descriptions_filename)

        with open(descriptions_path, 'r') as descriptions_file:
            for line in descriptions_file:
                description_breakdown = line.split(' ')
                description_breakdown.pop(0)
                description = ' '.join(description_breakdown).strip()
                descriptions.append(description)

        return descriptions

    def loadValidationGroundtruths(self):
        groundtruths = []

        groundtruths_filename = 'validation_ground_truth.txt'
        groundtruths_path = pathJoin(self.directory, '..', groundtruths_filename)

        with open(groundtruths_path, 'r') as groundtruths_file:
            for line in groundtruths_file:
                groundtruth_breakdown = line.split(' ')
                groundtruth_breakdown.pop(0)
                groundtruth = ' '.join(groundtruth_breakdown).strip()
                groundtruths.append(int(groundtruth))

        return groundtruths

    def loadClasses(self):
        classes = []

        classes_filename = 'synsets.txt'
        classes_path = pathJoin(self.directory, '..', classes_filename)

        with open(classes_path, 'r') as classes_file:
            for line in classes_file:
                classes.append(line.strip())

        return classes

BATCH_SIZE = 64

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     normalize
])

imagenet_dataset_path = os.path.join('datasets', 'imagenet')

# imagenet_train_dataset = ImageNetDataset(imagenet_dataset_path, transforms=test_transforms)
imagenet_val_dataset = ImageNetDataset(imagenet_dataset_path, split='val', transforms=test_transforms)
# imagenet_test_dataset = ImageNetDataset(imagenet_dataset_path, split='test')

# imagenet_train_loader = DataLoader(imagenet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=BATCH_SIZE, shuffle=False)
# imagenet_test_loader = DataLoader(imagenet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)


100%|██████████| 50000/50000 [00:01<00:00, 28969.44it/s]


In [4]:
indices = [i for i, s in enumerate(imagenet_val_dataset.descriptions) if 'cat' in s]
cats = [ imagenet_val_dataset.descriptions[i] for i in indices ]
for i in zip(indices, cats):
    print(i)

(143, 'oystercatcher')
(281, 'tabby cat')
(282, 'tiger cat')
(283, 'Persian cat')
(284, 'Siamese cat')
(358, 'European polecat')
(484, 'catamaran')


In [ ]:
cats = [ x for x in tqdm(imagenet_val_dataset) if x[2] == 281 ]

In [ ]:
toImage = torchvision.transforms.ToPILImage()

In [ ]:
total = len(cats)
row_size = 10

for index, cat in enumerate(cats):
    plt.figure(figsize=(100,100))
    plt.subplot(total, 1, index+1)
    plt.imshow(toImage(cat[1]))

In [ ]:
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50.eval()

toImage = torchvision.transforms.ToPILImage()

In [ ]:
current_index = 0

batch_accuracy = []
individual_accuracy = []

for index, batch in enumerate(imagenet_val_loader):
    resnet50.cuda()
    # batch
    batch_images = batch[1] # index 1 has the images
    b_output = resnet50(batch_images.cuda())
    _, b_pred = b_output.topk(1, 1, True, True)
#     print(batch[2])
#     print(b_pred.cpu().squeeze())
#     print(accuracy_score(batch[2], b_pred.cpu().squeeze()))
    batch_accuracy.append(accuracy_score(batch[2], b_pred.cpu().squeeze()))
    b_pred = b_pred[current_index] # prediction for first image
    
    # individually
    single_image = batch[1][current_index]
    single_image = single_image.unsqueeze(0) # include batch dimension
    i_output = resnet50(single_image.cuda())
    _, i_pred = i_output.topk(1, 1, True, True)
    
    print('{} (truth) -- {} (batch) -- {} (single)'.format(
        batch[3][current_index],
        imagenet_val_dataset.descriptions[b_pred.item()],
        imagenet_val_dataset.descriptions[i_pred.item()]
    ))
    
    print(batch_accuracy)

#     assert i_pred.item() == b_pred.item(), 'Predictions are not the same'
    if (index + 1) == 5:
        break

In [ ]:
x1 = torch.tensor([ 65, 970, 230, 809, 516,  57, 334, 415, 674, 332, 109, 286, 370, 757,
        595, 147, 108,  23, 478, 517, 334, 173, 948, 727,  23, 846, 270, 167,
         55, 858, 324, 573, 150, 981, 586, 887,  32, 398, 777,  74, 516, 756,
        129, 198, 256, 725, 565, 167, 717, 394,  92,  29, 844, 591, 358, 468,
        259, 994, 872, 588, 474, 183, 107,  46])
x2 = torch.tensor([ 65, 795, 230, 967, 520,  58, 334, 852, 674, 332, 109, 286, 370, 757,
        595, 147,  76,  21, 478, 517, 334, 173, 948, 727,  23, 846, 270, 166,
         55, 538, 324, 573, 360, 981, 586, 887,  26, 398, 777,  74, 431, 756,
        129, 198, 256, 505, 565, 162, 717, 395,  92,  29, 844, 591, 359, 468,
        259, 994, 840, 588, 474, 197, 107,  40])


y1 = torch.tensor([ 65, 970, 230, 809, 516,  57, 334, 415, 674, 332, 109, 286, 370, 757,
        595, 147, 108,  23, 478, 517, 334, 173, 948, 727,  23, 846, 270, 167,
         55, 858, 324, 573, 150, 981, 586, 887,  32, 398, 777,  74, 516, 756,
        129, 198, 256, 725, 565, 167, 717, 394,  92,  29, 844, 591, 358, 468,
        259, 994, 872, 588, 474, 183, 107,  46])
y2 = torch.tensor([ 65, 795, 231, 659, 431,  65, 334, 529, 532, 332, 109, 286, 370, 757,
        595, 147,  29,  21, 478, 517, 334, 354, 948, 727,  23, 846, 270, 166,
         64, 832, 324, 573,  33, 981, 586, 887,  26, 398, 529, 126, 431, 713,
        129, 196, 256, 968, 565, 180, 717,  69, 395,  29, 844, 591, 516, 468,
        259, 994, 840, 681, 841, 197,   5,  40])

print(accuracy_score(x1, x2))
print(accuracy_score(y1, y2))

print(torch.sum(x1 == y1))
print(torch.sum(x1 == x2))
print(torch.sum(y1 == y2))

In [ ]:
import os

values_directory = os.path.join('results', 'values')
values_files = os.listdir(values_directory)
for value_file in values_files:
    true_class, predicted_class, imagenet_val_index = value_file.split('.')[0].split('-')
    old_path = os.path.join(values_directory, value_file)
    new_path = os.path.join(values_directory, '{}-{}-{}.pickle'.format(imagenet_val_index, true_class, predicted_class))
    print('{} -> {}'.format(old_path, new_path))
#     os.rename(old_path, new_path)

In [ ]:
values_directory = os.path.join('results', 'plots')
values_files = os.listdir(values_directory)
for value_file in values_files:
    true_class, predicted_class, imagenet_val_index = value_file.split('.')[0].split('-')
    old_path = os.path.join(values_directory, value_file)
    new_path = os.path.join(values_directory, '{}-{}-{}.png'.format(imagenet_val_index, true_class, predicted_class))
    print('{} -> {}'.format(old_path, new_path))
#     os.rename(old_path, new_path)